In [1]:
import paho.mqtt.client
mqttbroker = "mqtt.local"   # set to None to disable
mqttspoken = "vocopi/spoken"
mqtttohear = "vocopi/tohear"
mqttclient = None
history = [ ]

def process(transcript):
    print("not overloaded yet")

mqttclient = paho.mqtt.client.Client(client_id="jcomputer")   # global object
def on_connect(client, userdata, flags, rc):
    print("on_connect", client, userdata, flags, rc)
def on_disconnect(client, userdata, rc):
    print("on_disconnect", client, userdata, rc)
def on_publish(client, userdata, mid):
    print("on_publish", client, userdata, mid)
def on_message(client, userdata, message):
    print("on_message", client, message, message.topic, message.payload)
    history.append(("received", message.payload.decode()))
    process(message.payload.decode())

mqttclient.on_connect = on_connect
mqttclient.on_publish = on_publish
mqttclient.on_disconnect = on_disconnect
mqttclient.on_message = on_message
print("made and attempting to connect", mqttclient)
mqttclient.connect(mqttbroker)
mqttclient.subscribe(mqttspoken)

mqttclient.publish(mqtttohear, "12,20 the computer is ready")

made and attempting to connect <paho.mqtt.client.Client object at 0x7f1130927828>
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 2


In [2]:
# eliza client (set to None if disabled)
elizabroker = "nodered.dynamicdevices.co.uk"
elizain = "eliza/jcomputer/in"
elizaout = "eliza/jcomputer/out"

elizaclient = paho.mqtt.client.Client(client_id="jj")
def on_ezmessage(client, userdata, message):
    print("on_message", client, message, message.topic, message.payload)
    say("40,50"+message.payload.decode())

elizaclient.on_connect = on_connect
elizaclient.on_publish = on_publish
elizaclient.on_disconnect = on_disconnect
elizaclient.on_message = on_ezmessage
print("made and attempting to connect", elizaclient)
elizaclient.connect(elizabroker)
elizaclient.subscribe(elizaout)
elizaclient.loop()


made and attempting to connect <paho.mqtt.client.Client object at 0x7f1130379780>


0

In [3]:
elizaclient.loop()


on_connect <paho.mqtt.client.Client object at 0x7f1130379780> None {'session present': 0} 0


0

In [4]:
#elizaclient = None
# mosquitto_sub -h mqtt.local -v -t vocopi/spoken

In [5]:
mqttclient.publish(mqtttohear, "40,50,hi there")

on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 3


In [6]:
mqttclient.loop()


on_connect <paho.mqtt.client.Client object at 0x7f1130927828> None {'session present': 0} 0


0

In [7]:
import re
def say(t):
    if not re.match("\d", t):
        t = "40,90 "+t
    mqttclient.publish(mqtttohear, t)
    
import time, datetime

tprevaction = 0
makestate = ""


coffeereport = [ "Coffee making report for "+datetime.datetime.now().isoformat(), "" ]

coffeesteps = [ "", "put the old filter into the compost", 
                "find a new coffee filter", 
                "add one measure of coffee to the filter", 
                "replace the filter funnel into machine", 
 #             "fill the jug with cold water", "pour water into the machine", "place jug on the hotplate", "turn on the machine", 
              "you are now done, wait for your coffee while I print the report" ]

def coffeemakingchecklist(words, say):
    global tprevaction, makestate
    print("cmc", words, makestate)
    if makestate == "commenced":
        say("Commencing the coffee making checklist.  Say check if done, or comment to add notes")
        makestate = 0
        words = ["check", "", "", ""]
        
    if words[0] == "repeat" or words[1] == "repeat":
        say("repeating item %d" % makestate)
        say(coffeesteps[makestate])
        return True
        
    if words[0] == "check":
        if makestate != 0:
            coffeereport.append("   "+datetime.datetime.now().isoformat()+"  check")
        makestate += 1
        say(coffeesteps[makestate])
        if makestate == len(coffeesteps) - 1:
            makestate = ""
            open("coffeereport.txt", "w").write("\n".join(coffeereport))
            return True
        coffeereport.append("")
        coffeereport.append(coffeesteps[makestate])
        return True
        
    if words[0] in ["close", "end", "stop"] and words[1] == "checklist":
        say("coffee making checklist aborted")
        makestate = ""
        return True
        
    if words[0] and type(makestate) == int:
        coffeereport.append("   "+datetime.datetime.now().isoformat()+" "+" ".join(words[:]))
        say("your comment on item %d has been noted" % makestate)
        return True
                
        
    say("You said "+" ".join(words)+" while making coffee")
    return True
    

elizaworking = False

def process(transcript):
    global tprevaction, makestate, elizaworking
    
    words = transcript.split()
    wc = len(words)
    tnow = time.time()
    dtime = tnow - tprevaction
    while len(words)<10:
        words.append("")
    print(words)
        
    if words[0].lower() == "eliza":
        if words[1] in ["stop", "end", "off"]:
            if elizaworking:
                elizaworking = False
                say("Eliza is gone")
            else:
                say("Eliza is not here")
        elif words[1] in ["start", "begin", "on"]:
            if elizaworking:
                say("Eliza is already here")
            elif elizaclient:
                elizaworking = True
                say("Eliza has arrived")
            else:
                say("Eliza cannot come")
        return True
    if elizaworking:
        elizaclient.publish(elizain, " ".join(words))
        return True
        
    if makestate:
        if makestate == "readytocommence":
            if dtime < 10 and words[0] == "coffee" and words[1] == "make":
                makestate = "commenced"
            else:
                makestate = ""
        coffeemakingchecklist(words, say)
        return True
        
    if words[0] == "coffee":
        if words[0] == "":
            say("say: coffee commands to get the command list")
            return True
        if words[1] == "commands":
            say("Available commands are: quantity, age, temperature, make")
            return True
        if words[1] == "quantity":
            say("There is half a pot left")
            return True
        if words[1] == "age":
            say("The coffee is 6 hours old")
            return True
        if words[1] == "temperature":
            say("The coffee is 36 degrees")
            return True
        if words[1] == "make":
            tprevaction = tnow
            if words[2] == "checklist":
                makestate = "commenced"
                coffeemakingchecklist(words, say)
                return True
            say("Say coffee make again to commence the coffee making checklist")
            makestate = "readytocommence"
            return True
                
            
    if wc == 1:
        say("one word")
    elif wc == 2:
        say("two words")
    elif wc == 3:
        say("three words")
    elif wc == 3:
        say("four words")
    return False


In [8]:
process("coffee commands")


['coffee', 'commands', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 4


True

In [9]:
mqttclient.loop(0.2)

0

In [ ]:
while True:
    mqttclient.loop(0.2)
    if elizaclient:
        elizaclient.loop(0.1)

on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b'the computer is ready said that'
['the', 'computer', 'is', 'ready', 'said', 'that', '', '', '', '']
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b'hi'
['hi', '', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 5
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d0b8> vocopi/spoken b'and then it gets into recording'
['and', 'then', 'it', 'gets', 'into', 'recording', '', '', '', '']
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d0b8> vocopi/spoken b'okay'
['okay', '', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 6
on_message <paho.mqtt.client

on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b'Eliza off'
['Eliza', 'off', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 16
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b'okay'
['okay', '', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 17
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d0b8> vocopi/spoken b'turn on the radio turn on windscreen wipers turn on the radio'
['turn', 'on', 'the', 'radio', 'turn', 'on', 'windscreen', 'wipers', 'turn', 'on', 'the', 'radio']
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d0b8> vocopi/spoken b'classical and it goes it looks like this music is a

on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b"I don't know it depends on what the question was"
['I', "don't", 'know', 'it', 'depends', 'on', 'what', 'the', 'question', 'was']
on_publish <paho.mqtt.client.Client object at 0x7f1130379780> None 30
on_message <paho.mqtt.client.Client object at 0x7f1130379780> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> eliza/jcomputer/out b'Do you wish to be able to know it depends on what the question was ?'
on_publish <paho.mqtt.client.Client object at 0x7f1130927828> None 34
on_message <paho.mqtt.client.Client object at 0x7f1130927828> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> vocopi/spoken b'yes please'
['yes', 'please', '', '', '', '', '', '', '', '']
on_publish <paho.mqtt.client.Client object at 0x7f1130379780> None 31
on_message <paho.mqtt.client.Client object at 0x7f1130379780> <paho.mqtt.client.MQTTMessage object at 0x7f113036d198> el